<img src="https://www.aepia.org/wp-content/uploads/2022/04/logo-aepia.png" width="200"/>

<img src="https://i1.rgstatic.net/ii/lab.file/AS%3A608742152355841%401522146847661_xl" width="300"/>

<img src="https://www.upo.es/cms1/export/sites/upo/comunicacion/imagenes/mic-upo-descargas/Marca-UPO-Horizontal.png" width="200"/>



# **Deep Learning: Image classification**
## Escuela de verano de Inteligencia Artificial 2023
#### José Fco. Torres Maldonado

jftormal@upo.es | datalab.upo.es/torres

---



In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from imutils import paths
import keras as keras
import os
import cv2
import zipfile
from sklearn.metrics import confusion_matrix

## 1. Dataset: 25000 fotografías de perros y gatos

Formato .zip<br/>
Nombradas como: cat.XXXX.jpg y dog.XXXX.jpg

In [ ]:
zip = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/EVIA2023/dataset.zip", 'r')
zip.extractall("/")
zip.close()

In [ ]:
print("Número de imágenes: ", len(os.listdir("/dataset")))
print("Contenido: ", os.listdir("/dataset"))

In [ ]:
img = cv2.imread('/dataset/dog.7182.jpg')

imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
imggray = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2GRAY)
img50x50 = cv2.resize(imggray, (50, 50))

fig, axs = plt.subplots(1, 4, figsize=(10, 10))
axs[0].imshow(img)
axs[1].imshow(imgrgb)
axs[2].imshow(imggray, cmap='gray')
axs[3].imshow(img50x50, cmap="gray")

plt.show()

## 2. Preprocesado

In [ ]:
data = []
labels = []

listaImagenes = list(paths.list_images('/dataset'))
print(listaImagenes[0:5])

In [ ]:
for (i, imgpath) in enumerate(listaImagenes):
    img = cv2.imread(imgpath)
    label = imgpath.split(os.path.sep)[-1].split(".")[0]
    features = image_as_vector(img)
    data.append((features, imgpath))
    labels.append(label)

In [ ]:
print("Total de imágenes:",len(data))
print("Registro:", data[0])
print("Imagen:", data[0][0])
print("Tamaño de cada imagen:", len(data[0][0])) #Equivale a 50x50x3 canales
print("Ruta:", data[0][1])
print("Etiqueta:", labels[0])

In [ ]:
imagesdata = [dat[0] for dat in data]
imagespaths = [dat[1] for dat in data]

print(imagesdata[0])
print(imagespaths[0])

In [ ]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

imgdata = np.array(imagesdata) / 255.0
labels = np_utils.to_categorical(labels,2)

In [ ]:
print("Imagen de ejemplo:",imgdata[0])
print("Etiqueta de ejemplo:",labels[0])

In [ ]:
(X_train, X_test, y_train, y_test, trainingpaths, testpaths) = train_test_split(imgdata, labels, imagespaths, test_size=0.30, random_state=42)

In [ ]:
print("Tamaño del training:", len(X_train))
print("Tamaño del test:", len(X_test))

## 3. Modelos Deep Learning

### 3.1 Convolutional Neural Network (CNN)

In [ ]:
cnnmodel.summary()

# Gráfica de la pérdida durante el entrenamiento
plt.figure(figsize=(12, 6))
plt.plot(cnnhistory.history['loss'])
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
(loss, accuracy) = cnnmodel.evaluate(X_test, y_test, batch_size=128, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy*100))

In [ ]:
probabilities = cnnmodel.predict(X_test)
predictions = np.round(np.argmax(probabilities, axis=1)).astype(int)

In [ ]:
real = np.argmax(y_test, axis=1) # y_test contiene por cada imagen un vector [X,Y]. Si X vale 1 -> gato; si Y vale 1 -> perro. Esta línea se queda con el índice para saber si es clase 0 (gato) o clase 1 (perro). codificación ONE-HOT

## Análisis de las predicciones

In [ ]:
def mostrar_resultado(idImagen):
    clase_real = "gato" if real[idImagen] == 0 else "perro"
    clase_predicha = "gato" if predictions[idImagen] == 0 else "perro"
    rutabase = "/dataset"
    fullpath = testpaths[idImagen]
    image = cv2.imread(fullpath)
    imgrgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(imgrgb)
    plt.axis('off')
    plt.show()
    print("CLASE REAL: \t", clase_real)
    print("CLASE PREDICHA:\t", clase_predicha)
    return fullpath

In [ ]:
mostrar_resultado(1444)

In [ ]:
# Calcular la exactitud (accuracy)
accuracy = accuracy_score(real, predictions)
print("ACC (%): ", accuracy*100)

# Calcular la precisión (precision)
precision = precision_score(real, predictions)
print("precision : ", precision)

# Calcular la exhaustividad (recall)
recall = recall_score(real, predictions)
print("recall : ", recall)

# Calcular la puntuación F1
f1 = f1_score(real, predictions)
print("F1 : ", f1)

# Calcular el área bajo la curva ROC (AUC-ROC)
auc_roc = roc_auc_score(real, predictions)
print("AUC : ", auc_roc)

In [ ]:
cm = confusion_matrix(real, predictions)
print(cm)